In [1]:
import cv2
import glob
import json
from tqdm import tqdm
import os

#path = '/home/shengtao/Data/censor_dat/test/'

#jfile = path + 'fc2_videos/fc2_video_2012.txt_ret.json'

#jdat = json.load(open(jfile,'r'))



In [2]:
def get_new_name(dst,filename, add_name):
    newfileName = dst+filename.split('.jpg')[0].split('/')[-1]+add_name+'.jpg'
    return newfileName
def draw_result(imgfile,ret,thr=0.3):
    txtMap = {'naked-body':'nbdy', 'naked-bottom':'nbot','underwear':'udw',
              'vagina':'vgna','hand':'hd','penis':'pns','breast':'brst','cum':'cum',
             'bra':'bra','dildo':'dld','butt':'bt'}
    img = cv2.imread(imgfile)
    #print(imgfile)
    if img is None:
        return img, True,''
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    isPorn=False
    add_name=[]
    for cls in ret.keys():
        for bb in ret[cls]:
            if bb[4] > thr:
                if not isPorn:
                    if cls != 'hand' and cls != 'tongue':
                        isPorn = True
                if txtMap[cls] not in add_name:
                    add_name.append(txtMap[cls])
                #add_name = add_name + txtMap[cls]+'_'
                pt1= (int(bb[0]),int(bb[1]))
                pt2=(int(bb[2]), int(bb[3]))
                img=cv2.rectangle(img,pt1,pt2,(255,0,0),3)
                if cls in txtMap.keys():
                    newName = txtMap[cls]
                else:
                    newName = cls
                img=cv2.putText(img,newName+'|%.2f'%bb[4],(int(bb[0]+10),int(bb[1]+40)), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,255),1,cv2.LINE_AA)
    name_str=''
    for elm in add_name:
        name_str=name_str +'_'+ elm
    #name_str = name_str
    return img, isPorn,name_str

def visualize_result(jfile,dst='./tmp/',thr=0.3):
    if not os.path.exists(dst):
        os.mkdir(dst)
    jdat = json.load(open(jfile,'r'))
    keys = list(jdat.keys())
    #count =0 
    for key in tqdm(keys):
        imgfiles = list(jdat[key])
        for imgfile in imgfiles:
            try:
                img_ret,isPorn,add_name = draw_result(imgfile, jdat[key][imgfile],thr)
                if img_ret is not None:
                    img_ret = cv2.cvtColor(img_ret, cv2.COLOR_RGB2BGR)
                if isPorn:
                    ppath = dst+'pred_porn/'
                    if not os.path.exists(ppath):
                        os.mkdir(ppath)
                    newFileName = get_new_name(ppath,imgfile, add_name)
                else:
                    npath = dst+'pred_norm/'
                    if not os.path.exists(npath):
                        os.mkdir(npath)
                    newFileName = get_new_name(npath,imgfile, add_name)
            #print(newFileName)
                cv2.imwrite(newFileName,img_ret)
            except:
                continue
            #count  = count +1 
        
# jfiles = glob.glob(path+'fc2_videos/*_ret.json')  
# for jfile in jfiles:
#     dst = jfile.replace('_ret.json','_visual/')
#     visualize_result(jfile,dst+'/')

jfile = './porn_sel.txt_ret.json'
#dst = jfile.replace('_ret.json','_visual/')
dst = '/home/jovyan/fast-data/porn_sel_ret/'
visualize_result(jfile,dst+'/',thr=0.4)

100%|██████████| 1/1 [05:59<00:00, 359.27s/it]


In [36]:
a= '/asdfas/bja/j'
os.path.dirname(a)

'/asdfas/bja'